In [4]:
import os
import re
import io
import math
import numpy as np
import pandas as pd

log_dir = "./logs/fgm"   # 필요하면 여기만 바꿔

# 파일명 패턴: Opp_bumper..._Gap..._Bub..._SPD..._FGM_..._dur_...s.csv
log_pattern = re.compile(
    r"^Opp_bumper.*"
    r"_Gap[-\d.]+"
    r"_Bub[-\d.]+"
    r"_Clr[-\d.]+"
    r"_WW[-\d.]+"
    r"_AW[-\d.]+"
    r"_SW[-\d.]+"
    r"_HB[-\d.]+"
    r"_CT[-\d.]+"
    r"_SA[-\d.]+"
    r"_DBS[-\d.]+"
    r"_SPD[-\d.]+"
    r"_FGM_.*_dur_[-\d.]+s\.csv$"
)

all_files = [
    os.path.join(log_dir, f)
    for f in os.listdir(log_dir)
    if f.endswith(".csv") and log_pattern.match(f)
]

print(f"발견된 로그 파일 개수: {len(all_files)}")
for f in all_files[:5]:
    print(" -", os.path.basename(f))


발견된 로그 파일 개수: 98
 - Opp_bumper_v_2.csv_Gap1.0_Bub0.5_Clr0.50_WW0.7_AW6.5_SW0.06_HB2.0_CT0.20_SA0.6_DBS0.15_SPD5.0_FGM_1764847509_GOAL_dur_10.01s.csv
 - Opp_bumper_slow_0.5.csv_Gap1.0_Bub0.4_Clr0.45_WW0.7_AW8.0_SW0.05_HB1.5_CT0.20_SA0.6_DBS0.18_SPD5.5_FGM_1764846921_ABORT_dur_3.37s.csv
 - Opp_bumper_slow_1.csv_Gap1.0_Bub0.4_Clr0.45_WW0.7_AW8.0_SW0.05_HB1.5_CT0.20_SA0.6_DBS0.18_SPD5.5_FGM_1764847105_GOAL_dur_10.27s.csv
 - Opp_bumper_slow_0.5.csv_Gap1.3_Bub0.55_Clr0.60_WW0.5_AW5.0_SW0.10_HB2.5_CT0.30_SA0.4_DBS0.15_SPD3.5_FGM_1764846475_GOAL_dur_11.10s.csv
 - Opp_bumper_v_2.csv_Gap1.2_Bub0.5_Clr0.55_WW0.6_AW6.0_SW0.08_HB2.0_CT0.25_SA0.5_DBS0.12_SPD4.0_FGM_1764847321_GOAL_dur_9.84s.csv


In [5]:
def load_run_log(path: str):
    """
    하나의 run_logger csv 파일을
    - df_data: per-step 데이터 (t,x,y,...)
    - summary_dict: 맨 아래 요약 블록 (key,value)
    로 나눠서 반환.
    """
    with open(path, "r") as f:
        lines = f.readlines()
    
    # Summary 시작 라인 찾기
    summary_idx = None
    for i, line in enumerate(lines):
        if line.strip().startswith('--- Experiment Summary'):
            summary_idx = i
            break
    
    if summary_idx is None:
        # Summary 없는 경우 (비상용)
        data_text = "".join(lines)
        df = pd.read_csv(io.StringIO(data_text))
        summary = {}
    else:
        # 맨 위 ~ summary 직전 빈 줄까지만 데이터로 사용
        data_text = "".join(lines[:summary_idx-1])
        df = pd.read_csv(io.StringIO(data_text))
        
        # summary 부분은 "key,value" 형식
        summary_lines = [l.strip() for l in lines[summary_idx+1:] if l.strip()]
        summary = {}
        for l in summary_lines:
            if "," in l:
                k, v = l.split(",", 1)
                summary[k.strip()] = v.strip()
    
    return df, summary


In [6]:
def parse_params_from_filename(path: str):
    """
    파일 이름에서
      scenario, Gap, Bub, Clr, WW, AW, SW, HB, CT, SA, DBS, SPD, result, duration_from_name
    를 뽑아서 dict로 반환.
    """
    name = os.path.basename(path)
    
    # 파라미터 블럭 파싱
    param_pattern = re.compile(
        r"_Gap(?P<Gap>[-\d.]+)"
        r"_Bub(?P<Bub>[-\d.]+)"
        r"_Clr(?P<Clr>[-\d.]+)"
        r"_WW(?P<WW>[-\d.]+)"
        r"_AW(?P<AW>[-\d.]+)"
        r"_SW(?P<SW>[-\d.]+)"
        r"_HB(?P<HB>[-\d.]+)"
        r"_CT(?P<CT>[-\d.]+)"
        r"_SA(?P<SA>[-\d.]+)"
        r"_DBS(?P<DBS>[-\d.]+)"
        r"_SPD(?P<SPD>[-\d.]+)"
    )
    
    m = param_pattern.search(name)
    params = {}
    if m:
        for k, v in m.groupdict().items():
            try:
                params[k] = float(v)
            except ValueError:
                params[k] = None
    
    # scenario (Gap 앞부분 전체)
    scen = name.split("_Gap")[0]
    params["scenario"] = scen   # ex) Opp_bumper_slow_0.5.csv
    
    # 결과 태그 (GOAL / CRASH / STUCK / ABORT 등)
    m2 = re.search(r"_(GOAL|CRASH|STUCK|ABORT)_", name)
    if m2:
        params["result"] = m2.group(1)
    
    # 파일명에 있는 duration
    m3 = re.search(r"_dur_([0-9.]+)s\.csv$", name)
    if m3:
        try:
            params["duration_from_name"] = float(m3.group(1))
        except ValueError:
            params["duration_from_name"] = None
    
    return params


In [7]:
def parse_summary_types(summary: dict):
    """
    summary dict 의 value를 적당히 숫자/불리언으로 캐스팅.
    """
    out = {}
    for k, v in summary.items():
        v = v.strip()
        # 숫자 시도
        try:
            out[k] = float(v)
            continue
        except ValueError:
            pass
        
        # YES/NO -> 1/0
        if v.upper() in ("YES", "NO"):
            out[k] = 1 if v.upper() == "YES" else 0
        else:
            out[k] = v
    return out


In [8]:
def compute_metrics(df: pd.DataFrame, safe_dist: float = 0.5):
    """
    run_logger 데이터 df 에서
    - total_time
    - collision_flag
    - min_dist / unsafe_ratio
    - fgm_min_dist / fgm_unsafe_ratio / fgm_track_rms
    를 계산 (있으면).
    """
    metrics = {}
    
    # 총 시간
    if "t" in df.columns and len(df) > 1:
        metrics["total_time"] = float(df["t"].iloc[-1] - df["t"].iloc[0])
    else:
        metrics["total_time"] = 0.0
    
    # 충돌 플래그
    if "collision" in df.columns:
        metrics["collision_flag"] = int(df["collision"].max() > 0)
    else:
        metrics["collision_flag"] = 0
    
    # 전역 min_d / unsafe_ratio
    if "min_d" in df.columns:
        metrics["min_dist"] = float(df["min_d"].min())
        metrics["unsafe_ratio"] = float((df["min_d"] < safe_dist).mean())
    
    # FGM 전용
    if "planner" in df.columns and "fgm_min_d" in df.columns:
        mask_fgm = df["planner"].astype(str).str.contains("FGM")
        if mask_fgm.any():
            df_fgm = df[mask_fgm]
            metrics["fgm_min_dist"] = float(df_fgm["fgm_min_d"].min())
            metrics["fgm_unsafe_ratio"] = float((df_fgm["fgm_min_d"] < safe_dist).mean())
            if "fgm_track" in df_fgm.columns:
                metrics["fgm_track_rms"] = float(math.sqrt((df_fgm["fgm_track"] ** 2).mean()))
    
    return metrics


In [9]:
def analyze_log(path: str, safe_dist: float = 0.5):
    """
    하나의 로그 파일 경로를 받아서
    - 파일 이름 기반 파라미터
    - summary 블럭 내용
    - per-step metrics
    다 합쳐서 dict 로 리턴.
    """
    df, summary = load_run_log(path)
    params   = parse_params_from_filename(path)
    metrics  = compute_metrics(df, safe_dist=safe_dist)
    summary_parsed = parse_summary_types(summary)
    
    row = {}
    row.update(params)
    row.update(summary_parsed)
    row.update(metrics)
    row["log_path"] = path
    
    return row


In [10]:
rows = []
for p in all_files:
    row = analyze_log(p, safe_dist=0.5)
    rows.append(row)

summary_df = pd.DataFrame(rows)
print("정리된 행 개수:", len(summary_df))
summary_df.head()


정리된 행 개수: 98


,Gap,Bub,Clr,WW,AW,SW,HB,CT,SA,DBS,...,Min Dist (m),Unsafe Ratio,FGM Min Dist (m),FGM Unsafe Ratio,FGM Track RMS (m),total_time,collision_flag,min_dist,unsafe_ratio,log_path
0,1.0,0.50,0.50,0.7,6.5,0.06,2.0,0.20,0.6,0.15,...,0.0,1.0,1.000000e+09,0.0,0.0,10.012810,0,0.0,1.0,./logs/fgm\Opp_bumper_v_2.csv_Gap1.0_Bub0.5_Cl...
1,1.0,0.40,0.45,0.7,8.0,0.05,1.5,0.20,0.6,0.18,...,0.0,1.0,1.000000e+09,0.0,0.0,3.370424,0,0.0,1.0,./logs/fgm\Opp_bumper_slow_0.5.csv_Gap1.0_Bub0...
2,1.0,0.40,0.45,0.7,8.0,0.05,1.5,0.20,0.6,0.18,...,0.0,1.0,1.000000e+09,0.0,0.0,10.265178,0,0.0,1.0,./logs/fgm\Opp_bumper_slow_1.csv_Gap1.0_Bub0.4...
3,1.3,0.55,0.60,0.5,5.0,0.10,2.5,0.30,0.4,0.15,...,0.0,1.0,1.000000e+09,0.0,0.0,11.096407,0,0.0,1.0,./logs/fgm\Opp_bumper_slow_0.5.csv_Gap1.3_Bub0...
4,1.2,0.50,0.55,0.6,6.0,0.08,2.0,0.25,0.5,0.12,...,0.0,1.0,1.000000e+09,0.0,0.0,9.836125,0,0.0,1.0,./logs/fgm\Opp_bumper_v_2.csv_Gap1.2_Bub0.5_Cl...


In [11]:
def get_first_existing(row, candidates, default=np.nan):
    for c in candidates:
        if c in row and not pd.isna(row[c]):
            return row[c]
    return default


In [12]:
def compute_safety_comfort_score(row,
                                 d_safe=0.5,
                                 d_good=0.8,
                                 track_rms_bad=1.0,
                                 w_safety=0.6,
                                 w_progress=0.3,
                                 w_comfort=0.1):
    """
    progress는 아직 모르는 상태라 safety+comfort만 보고 임시 점수 계산.
    (progress는 나중에 duration min/max로 정규화해서 반영)
    """
    # 충돌 / 비정상 종료 체크
    result = str(row.get("result", "")).upper()
    collision_flag = 0
    if "Collision" in row:
        collision_flag = int(row["Collision"])
    elif "collision_flag" in row:
        collision_flag = int(row["collision_flag"])
    
    if collision_flag == 1 or result in ("CRASH", "STUCK", "ABORT"):
        return 0.0
    
    # ----- Safety -----
    fgm_min = get_first_existing(row, ["FGM Min Dist (m)", "fgm_min_dist", "Min Dist (m)", "min_dist"], default=np.nan)
    if math.isnan(fgm_min):
        fgm_min = d_safe
    
    fgm_unsafe = get_first_existing(row, ["FGM Unsafe Ratio", "fgm_unsafe_ratio", "Unsafe Ratio", "unsafe_ratio"], default=0.0)
    if math.isnan(fgm_unsafe):
        fgm_unsafe = 0.0
    
    # 최소 거리 점수
    Sd = (fgm_min - d_safe) / (d_good - d_safe) if d_good > d_safe else 1.0
    Sd = max(0.0, min(1.0, Sd))
    
    # Unsafe Ratio 점수
    Su = 1.0 - max(0.0, min(1.0, fgm_unsafe))
    
    S_safety = 0.7 * Sd + 0.3 * Su
    
    # ----- Comfort (트랙 RMS) -----
    track_rms = get_first_existing(row, ["FGM Track RMS (m)", "fgm_track_rms"], default=0.0)
    if math.isnan(track_rms):
        track_rms = 0.0
    
    S_track = 1.0 - max(0.0, min(1.0, track_rms / track_rms_bad))
    S_comfort = S_track
    
    # progress 제외 상태에서 임시 점수 (safety+comfort 비율만 유지)
    # w_progress는 일단 무시하고, (w_safety + w_comfort)만 사용
    w_sc = w_safety + w_comfort
    Score0 = 0.0
    if w_sc > 0:
        Score0 = (w_safety * S_safety + w_comfort * S_comfort) / w_sc
    return Score0 * 100.0


In [13]:
# 1) safety/comfort 기반 임시 점수 계산
summary_df["Score_tmp"] = summary_df.apply(compute_safety_comfort_score, axis=1)

# 2) duration_raw 칼럼 준비 (Summary > 파일명 순으로)
summary_df["duration_raw"] = summary_df.apply(
    lambda r: get_first_existing(r, ["Duration (s)", "duration_from_name"], default=np.nan),
    axis=1
)

# 3) duration min/max (충돌/ABORT 제외)
valid_mask = (summary_df["Score_tmp"] > 0) & (~summary_df["duration_raw"].isna())
if valid_mask.any():
    T_min = summary_df.loc[valid_mask, "duration_raw"].min()
    T_max = summary_df.loc[valid_mask, "duration_raw"].max()
else:
    T_min, T_max = np.nan, np.nan

def compute_progress(duration, T_min, T_max):
    if math.isnan(duration) or math.isnan(T_min) or math.isnan(T_max) or T_max <= T_min:
        return 0.0
    val = (T_max - duration) / (T_max - T_min)
    return max(0.0, min(1.0, val))

summary_df["S_progress"] = summary_df["duration_raw"].apply(
    lambda t: compute_progress(t, T_min, T_max)
)

def compute_final_score(row,
                        w_safety=0.6,
                        w_progress=0.3,
                        w_comfort=0.1):
    # 충돌 / 비정상 종료면 0점
    result = str(row.get("result", "")).upper()
    collision_flag = 0
    if "Collision" in row:
        collision_flag = int(row["Collision"])
    elif "collision_flag" in row:
        collision_flag = int(row["collision_flag"])
    if collision_flag == 1 or result in ("CRASH", "STUCK", "ABORT"):
        return 0.0
    
    # safety+comfort는 Score_tmp 에 이미 들어 있음 (0~100)
    Score_tmp = row["Score_tmp"]
    # progress는 0~1
    S_progress = row["S_progress"]
    
    # 안전+컴포트 점수 비율 = (1 - w_progress)
    # 최종: Score = (1 - w_progress)*Score_tmp + w_progress*(100 * S_progress)
    Score_final = (1.0 - w_progress) * Score_tmp + w_progress * (100.0 * S_progress)
    
    # 클립
    return max(0.0, min(100.0, Score_final))

summary_df["Score"] = summary_df.apply(compute_final_score, axis=1)

# 점수 높은 순으로 보기
summary_df_sorted = summary_df.sort_values("Score", ascending=False)
summary_df_sorted.head(10)[[
    "scenario", "result",
    "Gap","Bub","Clr","WW","AW","SW","HB","CT","SA","DBS","SPD",
    "Score",
    "FGM Min Dist (m)","FGM Unsafe Ratio","FGM Track RMS (m)",
    "Duration (s)","duration_from_name"
]]


,scenario,result,Gap,Bub,Clr,WW,AW,SW,HB,CT,SA,DBS,SPD,Score,FGM Min Dist (m),FGM Unsafe Ratio,FGM Track RMS (m),Duration (s),duration_from_name
11,Opp_bumper_slow_1.5.csv,GOAL,1.1,0.45,0.50,0.9,7.0,0.07,1.0,0.22,0.70,0.10,4.5,100.000000,1.000000e+09,0.0,0.0,1.55,1.55
71,Opp_bumper_slow_0.5.csv,GOAL,1.0,0.40,0.45,0.7,8.0,0.05,1.5,0.20,0.60,0.18,5.5,98.909091,1.000000e+09,0.0,0.0,2.53,2.53
61,Opp_bumper_slow_2.csv,GOAL,1.0,0.50,0.50,0.7,6.5,0.06,2.0,0.20,0.60,0.15,5.0,97.450835,1.000000e+09,0.0,0.0,3.84,3.84
93,Opp_bumper_v_4.csv,GOAL,1.1,0.45,0.50,0.9,7.0,0.07,1.0,0.22,0.70,0.10,4.5,96.526902,1.000000e+09,0.0,0.0,4.67,4.67
81,Opp_bumper_slow_0.5.csv,GOAL,1.3,0.55,0.60,0.5,5.0,0.10,2.5,0.30,0.40,0.15,3.5,95.959184,1.000000e+09,0.0,0.0,5.18,5.18
43,Opp_bumper_slow_1.csv,GOAL,1.3,0.55,0.60,0.5,5.0,0.10,2.5,0.30,0.40,0.15,3.5,95.435993,1.000000e+09,0.0,0.0,5.65,5.65
62,Opp_bumper_slow_2.csv,GOAL,1.2,0.50,0.55,0.6,6.0,0.08,2.0,0.25,0.50,0.12,4.0,92.218924,1.000000e+09,0.0,0.0,8.54,8.54
22,Opp_bumper_v_2.csv,GOAL,1.3,0.60,0.55,0.5,5.5,0.08,2.0,0.25,0.75,0.25,4.2,90.916512,1.000000e+09,0.0,0.0,9.71,9.71
67,Opp_bumper_v_5.csv,GOAL,1.0,0.40,0.45,0.7,8.0,0.05,1.5,0.20,0.60,0.18,5.5,90.794063,1.000000e+09,0.0,0.0,9.82,9.82
4,Opp_bumper_v_2.csv,GOAL,1.2,0.50,0.55,0.6,6.0,0.08,2.0,0.25,0.50,0.12,4.0,90.771800,1.000000e+09,0.0,0.0,9.84,9.84


In [ ]:
param_cols = ["Gap","Bub","Clr","WW","AW","SW","HB","CT","SA","DBS","SPD"]

grouped = (
    summary_df
    .groupby(param_cols)
    .agg(
        mean_score=("Score", "mean"),
        min_score=("Score", "min"),
        max_score=("Score", "max"),
        runs=("Score", "count"),
        crashes=("Collision", "sum") if "Collision" in summary_df.columns else ("collision_flag", "sum")
    )
    .reset_index()
    .sort_values("mean_score", ascending=False)
)

grouped


,Gap,Bub,Clr,WW,AW,SW,HB,CT,SA,DBS,SPD,mean_score,min_score,max_score,runs,crashes
2,1.1,0.45,0.50,0.9,7.0,0.07,1.0,0.22,0.70,0.10,4.5,92.378942,89.959184,100.000000,8,0
1,1.0,0.50,0.50,0.7,6.5,0.06,2.0,0.20,0.60,0.15,5.0,91.215677,89.803340,97.450835,8,0
4,1.2,0.50,0.60,0.6,6.0,0.09,3.5,0.40,0.50,0.12,4.0,89.957792,88.846011,90.426716,8,0
5,1.3,0.55,0.60,0.5,5.0,0.10,2.5,0.30,0.40,0.15,3.5,85.744854,0.000000,95.959184,21,0
6,1.3,0.60,0.55,0.5,5.5,0.08,2.0,0.25,0.75,0.25,4.2,75.808905,0.000000,90.916512,8,0
7,1.5,0.70,0.75,0.4,4.0,0.13,3.0,0.35,0.30,0.22,3.0,75.138033,0.000000,89.191095,10,0
3,1.2,0.50,0.55,0.6,6.0,0.08,2.0,0.25,0.50,0.12,4.0,73.996627,0.000000,92.218924,22,1
0,1.0,0.40,0.45,0.7,8.0,0.05,1.5,0.20,0.60,0.18,5.5,69.466533,0.000000,98.909091,13,0
